In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean
import sqlite3 as sq

# Archive #4 (?)

#### Procedure for reading the data

We used SQLite to process the file.

In [2]:
conn = sq.connect('yeast.sqlite')